<a href="https://colab.research.google.com/github/drrayislam/NLP/blob/master/Text_Mining_Word_Frequencey_data_uploaded_from_Excel_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Upload the Excel files containing URLs and keywords
uploaded_urls = files.upload()  # Upload the URLs Excel file
uploaded_keywords = files.upload()  # Upload the Keywords Excel file

Saving test_websites.xlsx to test_websites.xlsx


Saving test_keywords.xlsx to test_keywords.xlsx


In [29]:
import pandas as pd

# Load the uploaded Excel files into DataFrames
df_urls = pd.read_excel('test_websites.xlsx')  # Replace with the actual file name
df_keywords = pd.read_excel('test_keywords.xlsx')  # Replace with the actual file name


In [30]:
print(df_urls.head())


                               URL
0  https://www.washingtonpost.com/
1         https://www.nytimes.com/


In [31]:
print(df_keywords.head())


      Keyword
0        hunt
1  operations
2        down
3        food


In [32]:
# Extract URLs and keywords from DataFrames
websites_list = df_urls['URL'].tolist()  # Adjust the column name as needed
specific_words = df_keywords['Keyword'].tolist()  # Adjust the column name as needed


In [33]:
print(df_keywords.head())


      Keyword
0        hunt
1  operations
2        down
3        food


In [34]:
import requests
from bs4 import BeautifulSoup
from collections import Counter
from urllib.parse import urljoin

In [35]:
# Function to extract text and word frequency from a URL
def extract_word_frequency(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        words = text.split()
        words = [word.lower() for word in words]
        word_frequency = Counter(words)
        return word_frequency
    else:
        return Counter()  # Return an empty Counter if the page can't be accessed


In [ ]:
# Function to extract word frequencies from all associated pages of a website
def extract_word_frequencies_for_website(website_url):
    visited_urls = set()  # Track visited URLs to avoid duplicates

    def recursive_crawl(url, word_frequency_total):
        if url in visited_urls:
            return word_frequency_total
        visited_urls.add(url)

        # Extract word frequency from the current page
        word_frequency = extract_word_frequency(url)
        word_frequency_total += word_frequency

        # Print word frequency for the current page
        print(f'URL: {url}')
        for word in specific_words:
            print(f'The word "{word}" appears {word_frequency[word.lower()]} times on this page.')

        # Find and follow links on the current page
        soup = BeautifulSoup(requests.get(url).text, 'html.parser')
        for link in soup.find_all('a', href=True):
            absolute_link = urljoin(url, link['href'])
            if website_url in absolute_link:  # Check if the link is within the same website
                word_frequency_total = recursive_crawl(absolute_link, word_frequency_total)

        return word_frequency_total

    word_frequency_total = Counter()  # Initialize word_frequency_total
    word_frequency_total = recursive_crawl(website_url, word_frequency_total)

    # Print word frequency totals across all pages of the website
    print("\nWord Frequency Totals Across All Pages of the Website:")
    for word in specific_words:
        print(f'Total "{word}" frequency: {word_frequency_total[word.lower()]}')

# Extract word frequencies for all websites
for website_url in websites_list:
    print(f"\nExtracting Word Frequencies for Website: {website_url}\n")
    extract_word_frequencies_for_website(website_url)


Extracting Word Frequencies for Website: https://www.washingtonpost.com/

URL: https://www.washingtonpost.com/
The word "hunt" appears 0 times on this page.
The word "operations" appears 0 times on this page.
The word "down" appears 4 times on this page.
The word "food" appears 2 times on this page.
URL: https://www.washingtonpost.com/accessibility
The word "hunt" appears 0 times on this page.
The word "operations" appears 0 times on this page.
The word "down" appears 0 times on this page.
The word "food" appears 0 times on this page.
